In [4]:
import glob
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

In [14]:
BOTTLENECK_TENSOR_SIZE = 2048

BOTTLENECT_TENSOR_NAME = 'pool_3/_reshape:0'

JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'

MODEL_DIR = 'data'

MODEL_FILE = 'tensorflow_inception_graph.pb'

CACHE_DIR = '/tmp/bottleneck'

INPUT_DATA = os.path.join('data', 'flower_photos')

VALIDATION_PERCENTAGE = 10
TEST_PERCENTAGE = 10

LEARNING_RATE = 0.01
STEPS = 20000
BATCH = 100

In [15]:
def create_image_lists(testing_percentage, validation_percentage):
    result = {}
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    is_root_dir = True
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue
        
        extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
        file_list = []
        dir_name = os.path.basename(sub_dir)
        for extension in extensions:
            file_glob = os.path.join(INPUT_DATA, dir_name, '*.'+extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list:
            continue
        label_name = dir_name.lower()
        training_images = []
        testing_images = []
        validation_images = []
        for file_name in file_list:
            base_name = os.path.basename(file_name)
            chance = np.random.randint(100)
            if chance < validation_percentage:
                validation_images.append(base_name)
            elif chance < (validation_percentage + testing_percentage):
                testing_images.append(base_name)
            else:
                training_images.append(base_name)
                
        result[label_name] = {
            'dir': dir_name,
            'training': training_images,
            'testing': testing_images,
            'validation': validation_images
        }
    return result

def get_image_path(image_info_map, image_root_dir, label_name, index, category):
    label_info_map = image_info_map[label_name]
    category_list = label_info_map[category]
    mod_index = index % len(category_list)
    base_name = category_list[mod_index]
    sub_dir = label_info_map['dir']
    full_path = os.path.join(image_root_dir, sub_dir, base_name)
    return full_path

def get_bottleneck_path(image_info_map, label_name, index, category):
    return get_image_path(image_info_map, CACHE_DIR, label_name, index, category) + '.txt'

def run_bottleneck_on_image(sess, image_data, image_data_tensor, bottleneck_tensor):
    bottleneck_values = sess.run(bottleneck_tensor, {image_data_tensor: image_data})
    bottleneck_values = np.squeeze(bottleneck_values)
    return bottleneck_values

def get_or_create_bottleneck(sess, image_info_map, label_name, index, category, jpeg_data_tensor, bottleneck_tensor):
    label_info_map = image_info_map[label_name]
    sub_dir = label_info_map['dir']
    sub_dir_path = os.path.join(CACHE_DIR, sub_dir)
    if not os.path.exists(sub_dir_path):
        os.makedirs(sub_dir_path)
    bottleneck_path = get_bottleneck_path(image_info_map, label_name, index, category)
    if not os.path.exists(bottleneck_path):
        image_path = get_image_path(image_info_map, INPUT_DATA, label_name, index, category)
        image_data = gfile.FastGFile(image_path, 'rb').read()
        bottleneck_values = run_bottleneck_on_image(sess, image_data, jpeg_data_tensor, bottleneck_tensor)
        bottleneck_string = ','.join(str(x) for x in bottleneck_values)
        with open(bottleneck_path, 'w') as bottleneck_file:
            bottleneck_file.write(bottleneck_string)
    else:
        with open(bottleneck_path, 'r') as bottleneck_file:
            bottleneck_string = bottleneck_file.read()
        bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
    return bottleneck_values

def get_random_cached_bottlenecks(sess, n_classes, image_info_map, how_many, category,
                                 jpeg_data_tensor, bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    for _ in range(how_many):
        label_index = random.randrange(n_classes)
        label_name = list(image_info_map.keys())[label_index]
        image_index = random.randrange(65536)
        bottleneck = get_or_create_bottleneck(sess, image_info_map, label_name, image_index,
                                             category, jpeg_data_tensor, bottleneck_tensor)
        ground_truth = np.zeros(n_classes, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)
    return bottlenecks, ground_truths

def get_test_bottlenecks(sess, image_info_map, n_classes, jpeg_data_tensor, bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    label_name_list = list(image_info_map.keys())
    for label_index, label_name in enumerate(label_name_list):
        category = 'testing'
        for index, _ in enumerate(image_info_map[label_name][category]):
            bottleneck = get_or_create_bottleneck(sess, image_info_map, label_name, index, category,
                                                 jpeg_data_tensor, bottleneck_tensor)
            ground_truth = np.zeros(n_classes, dtype=np.float32)
            ground_truth[label_index] = 1.0
            bottlenecks.append(bottleneck)
            ground_truths.append(ground_truth)
    return bottlenecks, ground_truths

def main():
    image_info_map = create_image_lists(TEST_PERCENTAGE, VALIDATION_PERCENTAGE)
    n_classes = len(image_info_map.keys())
    with gfile.FastGFile(os.path.join(MODEL_DIR, MODEL_FILE), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    bottleneck_tensor, jpeg_data_tensor = tf.import_graph_def(graph_def, 
                                                              return_elements=[BOTTLENECT_TENSOR_NAME, 
                                                                              JPEG_DATA_TENSOR_NAME])
    bottleneck_input = tf.placeholder(tf.float32, [None, BOTTLENECK_TENSOR_SIZE], name='BottleneckInputPlaceholder')
    ground_truth_input = tf.placeholder(tf.float32, [None, n_classes], name='GroundTruthInput')
    with tf.name_scope('final_training_ops'):
        weights = tf.Variable(tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, n_classes], stddev=0.1))
        biases = tf.Variable(tf.zeros([n_classes]))
        logits = tf.matmul(bottleneck_input, weights) + biases
        final_tensor = tf.nn.softmax(logits)
        
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=ground_truth_input))
    train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
    
    with tf.name_scope('evaluation'):
        correct_prediction = tf.equal(tf.argmax(final_tensor, 1), tf.argmax(ground_truth_input, 1))
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        for i in range(STEPS):
            train_bottlenecks, train_ground_truth = \
                get_random_cached_bottlenecks(sess, n_classes, image_info_map, BATCH, 'training',
                                             jpeg_data_tensor, bottleneck_tensor)
            sess.run(train_step, feed_dict={bottleneck_input: train_bottlenecks,
                                           ground_truth_input: train_ground_truth})
            if i % 100 == 0 or i + 1 == STEPS:
                validation_bottlenecks, validation_ground_truth = \
                    get_random_cached_bottlenecks(sess, n_classes, image_info_map, BATCH,
                                                 'validation', jpeg_data_tensor, bottleneck_tensor)
                validation_accuracy = sess.run(evaluation_step, feed_dict={bottleneck_input: validation_bottlenecks,
                                                                          ground_truth_input: validation_ground_truth})
                print('Step %d: Validation accuracy on random samples %d examples = %.1f%%' % 
                        (i+1, BATCH, validation_accuracy * 100))
                
                test_bottlenecks, test_ground_truth = \
                    get_test_bottlenecks(sess, image_info_map, n_classes, jpeg_data_tensor, bottleneck_tensor)
                test_accuracy = sess.run(evaluation_step, feed_dict={bottleneck_input: test_bottlenecks,
                                                                    ground_truth_input: test_ground_truth})
                print('Test accuracy = %.1f%%' % (test_accuracy * 100))
                
main()

Step 1: Validation accuracy on random samples 100 examples = 22.0%
Test accuracy = 18.5%
Step 101: Validation accuracy on random samples 100 examples = 76.0%
Test accuracy = 68.7%
Step 201: Validation accuracy on random samples 100 examples = 74.0%
Test accuracy = 78.6%
Step 301: Validation accuracy on random samples 100 examples = 79.0%
Test accuracy = 80.3%
Step 401: Validation accuracy on random samples 100 examples = 84.0%
Test accuracy = 81.2%
Step 501: Validation accuracy on random samples 100 examples = 86.0%
Test accuracy = 80.1%
Step 601: Validation accuracy on random samples 100 examples = 86.0%
Test accuracy = 81.5%
Step 701: Validation accuracy on random samples 100 examples = 83.0%
Test accuracy = 82.6%
Step 801: Validation accuracy on random samples 100 examples = 81.0%
Test accuracy = 83.8%
Step 901: Validation accuracy on random samples 100 examples = 79.0%
Test accuracy = 84.3%
Step 1001: Validation accuracy on random samples 100 examples = 85.0%
Test accuracy = 84.9%


Test accuracy = 90.9%
Step 9001: Validation accuracy on random samples 100 examples = 90.0%
Test accuracy = 90.9%
Step 9101: Validation accuracy on random samples 100 examples = 94.0%
Test accuracy = 90.6%
Step 9201: Validation accuracy on random samples 100 examples = 97.0%
Test accuracy = 90.9%
Step 9301: Validation accuracy on random samples 100 examples = 89.0%
Test accuracy = 90.6%
Step 9401: Validation accuracy on random samples 100 examples = 96.0%
Test accuracy = 90.6%
Step 9501: Validation accuracy on random samples 100 examples = 89.0%
Test accuracy = 90.3%
Step 9601: Validation accuracy on random samples 100 examples = 96.0%
Test accuracy = 90.6%
Step 9701: Validation accuracy on random samples 100 examples = 87.0%
Test accuracy = 90.6%
Step 9801: Validation accuracy on random samples 100 examples = 83.0%
Test accuracy = 90.6%
Step 9901: Validation accuracy on random samples 100 examples = 85.0%
Test accuracy = 90.6%
Step 10001: Validation accuracy on random samples 100 exam

Step 17801: Validation accuracy on random samples 100 examples = 96.0%
Test accuracy = 90.6%
Step 17901: Validation accuracy on random samples 100 examples = 89.0%
Test accuracy = 90.9%
Step 18001: Validation accuracy on random samples 100 examples = 90.0%
Test accuracy = 90.9%
Step 18101: Validation accuracy on random samples 100 examples = 95.0%
Test accuracy = 90.9%
Step 18201: Validation accuracy on random samples 100 examples = 91.0%
Test accuracy = 90.9%
Step 18301: Validation accuracy on random samples 100 examples = 93.0%
Test accuracy = 90.9%
Step 18401: Validation accuracy on random samples 100 examples = 91.0%
Test accuracy = 90.9%
Step 18501: Validation accuracy on random samples 100 examples = 92.0%
Test accuracy = 90.9%
Step 18601: Validation accuracy on random samples 100 examples = 87.0%
Test accuracy = 90.9%
Step 18701: Validation accuracy on random samples 100 examples = 98.0%
Test accuracy = 90.9%
Step 18801: Validation accuracy on random samples 100 examples = 92.0%